# WhisperX Cloud API Server 🚀 (Universal)

**支持平台:** Google Colab | Kaggle | 本地 GPU 服务器

这个 Notebook 自动检测运行环境并部署 WhisperX API 服务。

## 🎯 特性
- ✅ 自动检测环境 (Colab/Kaggle/Local)
- ✅ 自动 GPU/CPU 检测
- ✅ 内置 ngrok 隧道（公共 URL）
- ✅ 支持多语言
- ✅ 单词级时间戳对齐
- ✅ 可选说话人分离

## 📋 使用步骤
1. **设置 GPU**: 在 Colab/Kaggle 中启用 GPU
2. **配置 ngrok**: 在下面设置 ngrok token
3. **运行所有单元格**: 点击 Runtime → Run all
4. **复制 URL**: 将生成的公共 URL 复制到 VideoLingo 配置

---

## 0️⃣ 环境配置

In [ ]:
# ============================================
# CONFIGURATION - 在这里配置你的参数
# ============================================

# ngrok 认证令牌 (必需 - 从 https://dashboard.ngrok.com 获取)
NGROK_AUTH_TOKEN = ""

# API 服务端口
SERVER_PORT = 8000

# 默认 Whisper 模型
DEFAULT_MODEL = "large-v3"

# 是否启用说话人分离 (需要更多 GPU 内存)
ENABLE_DIARIZATION = False

# HuggingFace 镜像 (中国大陆用户可设置为 "https://hf-mirror.com")
HF_ENDPOINT = "https://huggingface.co"

print("⚙️ Configuration loaded")
print(f"   Model: {DEFAULT_MODEL}")
print(f"   Port: {SERVER_PORT}")
print(f"   HF Endpoint: {HF_ENDPOINT}")

## 1️⃣ 环境检测与设置

In [ ]:
# ============================================
# 检测运行环境
# ============================================

import sys
import os

# 检测平台
IN_COLAB = 'google.colab' in sys.modules
IN_KAGGLE = os.path.exists('/kaggle')
IN_LOCAL = not IN_COLAB and not IN_KAGGLE

print("🔍 Environment Detection:")
print(f"   Google Colab: {IN_COLAB}")
print(f"   Kaggle: {IN_KAGGLE}")
print(f"   Local: {IN_LOCAL}")

# 设置 HuggingFace 端点
os.environ['HF_ENDPOINT'] = HF_ENDPOINT

# Kaggle 特殊处理
if IN_KAGGLE:
    print("\n📌 Kaggle Instructions:")
    print("   1. Settings → Accelerator → GPU T4 x2")
    print("   2. Internet must be ON for ngrok")

## 2️⃣ GPU 检查

In [ ]:
# ============================================
# 检查 GPU 可用性
# ============================================

import subprocess

result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
print(result.stdout)

if result.returncode != 0:
    print("\n⚠️  WARNING: No GPU detected or nvidia-smi not found")
    print("   Server will run in CPU mode (slow)\n")
else:
    print("\n✅ GPU detected!\n")

## 3️⃣ 安装依赖

In [ ]:
# ============================================
# 安装依赖包
# ============================================

import subprocess
import sys

print("📦 Installing dependencies...\n")

# 核心依赖
packages = [
    "fastapi",
    "uvicorn[standard]",
    "python-multipart",
    "pydantic",
    "requests",
    "pyngrok",
]

# Kaggle 需要 nest_asyncio
if IN_KAGGLE:
    packages.append("nest_asyncio")

# 安装
for pkg in packages:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])
    print(f"   ✅ {pkg}")

# 安装 PyTorch (CUDA 11.8)
print("\n📦 Installing PyTorch with CUDA support...")
subprocess.check_call([
    sys.executable, "-m", "pip", "install", "-q",
    "torch==2.0.0+cu118", "torchaudio==2.0.0+cu118",
    "--extra-index-url", "https://download.pytorch.org/whl/cu118"
])
print("   ✅ PyTorch")

# 安装 WhisperX
print("\n📦 Installing WhisperX...")
subprocess.check_call([
    sys.executable, "-m", "pip", "install", "-q",
    "whisperx@git+https://github.com/m-bain/whisperx.git@7307306a9d8dd0d261e588cc933322454f853853"
])
print("   ✅ WhisperX")

# 可选：说话人分离
if ENABLE_DIARIZATION:
    print("\n📦 Installing speaker diarization...")
    subprocess.check_call([
        sys.executable, "-m", "pip", "install", "-q",
        "pyannote.audio==3.1.1"
    ])
    print("   ✅ pyannote.audio")

print("\n✅ All dependencies installed!")

## 4️⃣ 创建 API 服务器

In [ ]:
# ============================================
# 生成服务器代码
# ============================================

server_code = '''
import os
import tempfile
import warnings
import time
import platform
from typing import Optional
from contextlib import asynccontextmanager

# Environment-specific imports
try:
    import nest_asyncio
    nest_asyncio.apply()
except ImportError:
    pass

import torch
import whisperx
import librosa
from fastapi import FastAPI, File, UploadFile, HTTPException, Form
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import uvicorn

warnings.filterwarnings("ignore")

# Global state
model_cache = {}
device = None
compute_type = None
platform_name = None

class TranscriptionResponse(BaseModel):
    success: bool
    language: str
    segments: list
    word_segments: Optional[list] = None
    speakers: Optional[list] = None
    processing_time: float
    device: str
    model: str
    platform: str

@asynccontextmanager
async def lifespan(app: FastAPI):
    global device, compute_type, platform_name
    
    # Detect platform
    if os.path.exists('/kaggle'):
        platform_name = "Kaggle"
    elif 'google.colab' in str(globals()):
        platform_name = "Colab"
    else:
        platform_name = "Local"
    
    # Detect device
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if device == "cuda":
        gpu_mem = torch.cuda.get_device_properties(0).total_memory / (1024**3)
        compute_type = "float16" if torch.cuda.is_bf16_supported() else "int8"
        print(f"🚀 Platform: {platform_name}")
        print(f"🚀 GPU: {torch.cuda.get_device_name(0)}")
        print(f"💾 GPU Memory: {gpu_mem:.2f} GB")
        print(f"⚙️  Compute type: {compute_type}")
    else:
        compute_type = "int8"
        print(f"⚠️  CPU mode on {platform_name}")
    
    yield
    
    # Cleanup
    print("\n🧹 Cleaning up...")
    model_cache.clear()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

app = FastAPI(
    title="WhisperX Cloud API",
    description="Universal WhisperX ASR service for VideoLingo",
    version="2.0.0",
    lifespan=lifespan
)

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

def get_or_load_model(model_name: str, language: Optional[str] = None):
    """Load or get cached model"""
    cache_key = f"{model_name}_{language}_{compute_type}"
    
    if cache_key not in model_cache:
        print(f"\n📥 Loading model: {model_name}...")
        
        vad_options = {"vad_onset": 0.500, "vad_offset": 0.363}
        asr_options = {"temperatures": [0], "initial_prompt": ""}
        
        model = whisperx.load_model(
            model_name,
            device,
            compute_type=compute_type,
            language=language,
            vad_options=vad_options,
            asr_options=asr_options
        )
        model_cache[cache_key] = model
        print(f"✅ Model loaded: {model_name}")
    
    return model_cache[cache_key]

def process_audio(audio_bytes: bytes):
    """Process uploaded audio"""
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp:
        tmp.write(audio_bytes)
        tmp_path = tmp.name
    
    try:
        audio, sr = librosa.load(tmp_path, sr=16000, mono=True)
        return audio
    finally:
        os.unlink(tmp_path)

# ==================== API Endpoints ====================

@app.get("/")
async def health_check():
    """Health check endpoint"""
    gpu_mem = None
    if torch.cuda.is_available():
        gpu_mem = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    
    return {
        "status": "healthy",
        "platform": platform_name,
        "device": device,
        "cuda_available": torch.cuda.is_available(),
        "gpu_memory_gb": gpu_mem,
        "models_cached": len(model_cache),
        "model_keys": list(model_cache.keys())
    }

@app.post("/transcribe", response_model=TranscriptionResponse)
async def transcribe(
    audio: UploadFile = File(..., description="Audio file to transcribe"),
    language: Optional[str] = Form(None, description="Language code (e.g., en, zh, ja)"),
    model: str = Form("large-v3", description="Whisper model name"),
    batch_size: Optional[int] = Form(None, description="Batch size (auto if not set)"),
    align: bool = Form(True, description="Enable word-level alignment"),
    speaker_diarization: bool = Form(False, description="Enable speaker diarization")
):
    """
    Transcribe audio file
    
    Returns segments with word-level timestamps
    """
    start_time = time.time()
    
    # Auto batch size
    if batch_size is None:
        if device == "cuda":
            gpu_mem = torch.cuda.get_device_properties(0).total_memory / (1024**3)
            batch_size = 16 if gpu_mem > 8 else 4
        else:
            batch_size = 1
    
    try:
        print(f"\n🎯 Transcribing: {audio.filename}")
        print(f"   Language: {language or 'auto-detect'}")
        print(f"   Model: {model}")
        print(f"   Batch size: {batch_size}")
        
        # Read and process audio
        audio_bytes = await audio.read()
        audio_array = process_audio(audio_bytes)
        
        # Load model and transcribe
        whisper_model = get_or_load_model(model, language)
        result = whisper_model.transcribe(audio_array, batch_size=batch_size)
        
        detected_language = result.get("language", language or "unknown")
        segments = result.get("segments", [])
        
        print(f"\n✅ Transcription done: {len(segments)} segments")
        
        # Word-level alignment
        word_segments = None
        if align and segments:
            print("🔄 Aligning words...")
            align_model, align_metadata = whisperx.load_align_model(
                language_code=detected_language,
                device=device
            )
            result_aligned = whisperx.align(
                segments,
                align_model,
                align_metadata,
                audio_array,
                device,
                return_char_alignments=False
            )
            segments = result_aligned.get("segments", [])
            word_segments = result_aligned.get("word_segments", [])
            del align_model
            torch.cuda.empty_cache()
            print(f"✅ Alignment done: {len(word_segments or [])} words")
        
        # Speaker diarization
        speakers = None
        if speaker_diarization:
            print("🎭 Performing speaker diarization...")
            try:
                diarize_model = whisperx.DiarizationPipeline(
                    model_name="pyannote/speaker-diarization-3.1",
                    device=device
                )
                diarize_segments = diarize_model(audio_array)
                result_diarized = whisperx.assign_word_speakers(diarize_segments, {"segments": segments})
                segments = result_diarized.get("segments", [])
                speakers = list(set(seg.get("speaker", "UNKNOWN") for seg in segments if "speaker" in seg))
                print(f"✅ Diarization done: {len(speakers)} speakers")
            except Exception as e:
                print(f"⚠️  Diarization failed: {e}")
        
        processing_time = time.time() - start_time
        
        return TranscriptionResponse(
            success=True,
            language=detected_language,
            segments=segments,
            word_segments=word_segments,
            speakers=speakers,
            processing_time=processing_time,
            device=device,
            model=model,
            platform=platform_name
        )
        
    except Exception as e:
        import traceback
        error_msg = str(e)
        print(f"\n❌ Error: {error_msg}")
        print(traceback.format_exc())
        raise HTTPException(status_code=500, detail=error_msg)
    finally:
        torch.cuda.empty_cache()

@app.delete("/cache")
async def clear_cache():
    """Clear model cache to free GPU memory"""
    global model_cache
    cleared_count = len(model_cache)
    model_cache.clear()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    return {
        "status": "Cache cleared",
        "models_cleared": cleared_count
    }

@app.get("/stats")
async def get_stats():
    """Get server statistics"""
    gpu_stats = {}
    if torch.cuda.is_available():
        gpu_stats = {
            "name": torch.cuda.get_device_name(0),
            "total_memory_gb": torch.cuda.get_device_properties(0).total_memory / (1024**3),
            "allocated_memory_gb": torch.cuda.memory_allocated() / (1024**3),
            "reserved_memory_gb": torch.cuda.memory_reserved() / (1024**3),
        }
    
    return {
        "platform": platform_name,
        "device": device,
        "models_cached": len(model_cache),
        "gpu": gpu_stats
    }

if __name__ == "__main__":
    port = int(os.environ.get("PORT", 8000))
    host = os.environ.get("HOST", "0.0.0.0")
    uvicorn.run(app, host=host, port=port)
'''

# 写入文件
with open('whisperx_server.py', 'w', encoding='utf-8') as f:
    f.write(server_code)

print("✅ Server code created: whisperx_server.py")
print(f"   Size: {len(server_code)} bytes")

## 5️⃣ 配置 ngrok

In [ ]:
# ============================================
# 配置 ngrok
# ============================================

from pyngrok import ngrok

if not NGROK_AUTH_TOKEN:
    print("❌ ERROR: NGROK_AUTH_TOKEN is not set!")
    print("\n请按以下步骤获取:")
    print("1. 访问 https://dashboard.ngrok.com/signup")
    print("2. 注册并登录")
    print("3. 访问 https://dashboard.ngrok.com/get-started/your-authtoken")
    print("4. 复制 token 到上面的配置单元格")
    raise ValueError("NGROK_AUTH_TOKEN required")

ngrok.set_auth_token(NGROK_AUTH_TOKEN)
print("✅ ngrok configured successfully!")

## 6️⃣ 启动服务器

In [ ]:
# ============================================
# 启动 API 服务器和 ngrok 隧道
# ============================================

import subprocess
import time
import os
import signal

# 清理旧进程
print("🧹 Cleaning up old processes...")
subprocess.run("pkill -f whisperx_server.py 2>/dev/null || true", shell=True)
subprocess.run("pkill -f ngrok 2>/dev/null || true", shell=True)
time.sleep(2)

# 关闭现有 ngrok 隧道
try:
    ngrok.kill()
except:
    pass

# 设置环境变量
os.environ['PORT'] = str(SERVER_PORT)
os.environ['HF_ENDPOINT'] = HF_ENDPOINT

# 启动服务器
print("\n🚀 Starting WhisperX API server...")
server_process = subprocess.Popen(
    [sys.executable, 'whisperx_server.py'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True,
    preexec_fn=os.setsid if hasattr(os, 'setsid') else None
)

# 等待服务器启动
print("⏳ Waiting for server to start (10s)...")
time.sleep(10)

# 启动 ngrok
print("\n🌐 Creating ngrok tunnel...")
try:
    public_url = ngrok.connect(SERVER_PORT, "http")
    
    print("\n" + "="*60)
    print("✅ SERVER IS RUNNING!")
    print("="*60)
    print(f"\n🌐 Public URL: {public_url}")
    print(f"🔗 API Endpoint: {public_url}/transcribe")
    print(f"🏥 Health Check: {public_url}/")
    print(f"📊 Stats: {public_url}/stats")
    print("\n" + "="*60)
    print("📋 Copy the Public URL to VideoLingo config!")
    print("="*60 + "\n")
    
    # 保存 URL
    with open('server_url.txt', 'w') as f:
        f.write(str(public_url))
    
    SERVER_URL = str(public_url)
    
except Exception as e:
    print(f"\n❌ Error starting ngrok: {e}")
    print("\nTrying local tunnel alternative...")
    SERVER_URL = None

## 7️⃣ 测试 API

In [ ]:
# ============================================
# 测试 API 连接
# ============================================

import requests

# 读取 URL
try:
    with open('server_url.txt', 'r') as f:
        API_URL = f.read().strip()
except:
    API_URL = f"http://localhost:{SERVER_PORT}"

print(f"Testing: {API_URL}")
print("-"*50)

# 健康检查
try:
    response = requests.get(f"{API_URL}/", timeout=10)
    data = response.json()
    
    print("✅ Health Check PASSED\n")
    print(f"   Status: {data.get('status')}")
    print(f"   Platform: {data.get('platform')}")
    print(f"   Device: {data.get('device')}")
    if data.get('gpu_memory_gb'):
        print(f"   GPU Memory: {data['gpu_memory_gb']:.2f} GB")
    print(f"   Models Cached: {data.get('models_cached', 0)}")
    
except Exception as e:
    print(f"❌ Health check failed: {e}")
    print("\nTroubleshooting:")
    print("1. Check if server is running (previous cell)")
    print("2. Try waiting a bit longer and re-run")
    print("3. Check ngrok status")

## 8️⃣ 保持运行 (⚠️ 不要停止此单元格!)

In [ ]:
# ============================================
# 保持服务器运行
# ============================================

print("💓 Server is running...\n")
print("Press STOP button in toolbar to stop\n")

try:
    count = 0
    while True:
        time.sleep(30)
        count += 1
        
        # 每分钟检查一次健康状态
        if count % 2 == 0:
            try:
                r = requests.get(f"{API_URL}/", timeout=5)
                if r.status_code == 200:
                    print(f"✅ {time.strftime('%H:%M:%S')} - Server healthy")
                else:
                    print(f"⚠️  {time.strftime('%H:%M:%S')} - Status: {r.status_code}")
            except:
                print(f"⚠️  {time.strftime('%H:%M:%S')} - Health check failed")
        else:
            print(f"💓 {time.strftime('%H:%M:%S')} - Running...")
            
except KeyboardInterrupt:
    print("\n\n🛑 Stopping server...")
    ngrok.kill()
    if server_process:
        server_process.terminate()
        server_process.wait()
    print("✅ Server stopped")

## 📚 VideoLingo 配置指南

服务器运行后，将 URL 配置到 VideoLingo:

### 方法 1: 修改 config.yaml
```yaml
whisper:
  runtime: 'cloud'
  whisperX_cloud_url: 'https://xxxx.ngrok-free.app'  # 你的 ngrok URL
```

### 方法 2: 环境变量
```bash
export WHISPERX_CLOUD_URL='https://xxxx.ngrok-free.app'
```

### 方法 3: VideoLingo 客户端
使用 `whisperx_cloud_client.py` 中的 `WhisperXCloudClient` 类

## 🔧 故障排除

### 1. ngrok 连接失败
- 检查 token 是否正确
- 确认网络连接（Kaggle 需要 Internet ON）
- 尝试重新运行第 5、6 单元格

### 2. GPU 未检测到
- Colab: Runtime → Change runtime type → GPU
- Kaggle: Settings → Accelerator → GPU T4 x2

### 3. 模型下载慢
- 设置 HF_ENDPOINT 为镜像站（中国大陆）
- 或等待首次下载完成（约 3GB）

### 4. 显存不足
- 使用较小模型: 'medium' 或 'small'
- 减小 batch_size
- 禁用 speaker_diarization